In [ ]:
!pip install -r requirements.txt

In [ ]:
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=186el6hPe81eA1YkS_2PU4S4ZbYzhNPjr' -O dataset.csv

# 1. Exploratory Data Analysis

In [ ]:
dataset = pd.read_csv('dataset.csv')

In [ ]:
dataset.head()

In [ ]:
dataset['sentiment'].hist()

We are mostly interested in negative comments, thus we assign 1 label to them.

In [ ]:
dataset['sentiment'] = dataset['sentiment'].map({'positive': 0,
                                                 'negative': 1})
dataset.head()

In [ ]:
reviews = dataset['review'].values.tolist()
reviews[:10]

# 2. Data preprocessing
## 2.1. Remove structures without semantic meaning

In [ ]:
import re

In [ ]:
reviews = [re.sub("<.+?>", "", rev) for rev in reviews]
reviews[:2]

In [ ]:
reviews = [re.sub('\S*\d+\S*', "", rev) for rev in reviews]
reviews[:2]

## 2.2. Remove capital letters

In [ ]:
reviews = [rev.lower() for rev in reviews]
reviews[:2]

## 2.3. Tokenization

In [ ]:
nltk.download('punkt')

reviews = [nltk.word_tokenize(rev) for rev in reviews]
reviews[:2]

## 2.4. Stopwords removal

In [ ]:
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
stopwords

In [ ]:
stopwords.extend([
    "'ll",
    "'re",
    "'m"
])

In [ ]:
stopwords

In [ ]:
reviews = [[token for token in rev if token not in stopwords] for rev in reviews]
reviews[:2]

## 2.5. Stemming

In [ ]:
stemmer = nltk.PorterStemmer()

In [ ]:
reviews[:2]

In [ ]:
from tqdm import tqdm

In [ ]:
reviews = [[stemmer.stem(word) for word in rev] for rev in tqdm(reviews, position=0, leave=True)]

In [ ]:
reviews[0]

## 2.6. Joining preprocessed words into full sentences

In [ ]:
reviews = [" ".join(rev) for rev in reviews]

## 2.7. Punctuation removal

In [ ]:
import string

string.punctuation

In [ ]:
punk_table = str.maketrans({key: None for key in string.punctuation})
punk_table

In [ ]:
reviews = [rev.translate(punk_table) for rev in reviews]
reviews[0]

## 2.8. Putting all toghether into TextCleaner

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.patterns = ["<.+?>", "\S*\d+\S*"]
        self.stopwords = nltk.corpus.stopwords.words('english') + ["'ll", "'re", "'m"]
        self.punk_table = str.maketrans({key: None for key in string.punctuation})
        self.stemmer = nltk.PorterStemmer()

    def fit(self, X, y=None):
        return self # unused method

    def transform(self, X):
        for pattern in self.patterns:
            X = [re.sub(pattern, '', rev) for rev in X]
        X = [rev.lower() for rev in X]
        X = [nltk.word_tokenize(rev) for rev in X]
        X = [[token for token in rev if token not in self.stopwords] for rev in X]
        X = [[self.stemmer.stem(token) for token in rev] for rev in X]
        X = [" ".join(rev) for rev in X]
        X = [rev.translate(self.punk_table) for rev in X]
        return X 

In [ ]:
text_cleaner = TextCleaner()
cleaned_reviews = text_cleaner.transform(dataset['review'].values.tolist())

In [ ]:
cleaned_reviews[:2]

## 2.9. Sentence vectorization using TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.3)

reviews_tfidf = tfidf.fit_transform(cleaned_reviews)
reviews_tfidf

In [ ]:
tfidf.get_feature_names()

In [ ]:
reviews_tfidf.getrow(0).todense().A.max()

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(reviews_tfidf.tocsr()[:200,:500].todense().A)
plt.xlabel('First 500 tokens')
plt.ylabel('First 200 reviews')

In [ ]:
f'{reviews_tfidf.data.nbytes / 1e6} Mb'

## 2.9. Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews_tfidf, dataset['sentiment'].values.ravel(), test_size=0.1, random_state=42)

In [ ]:
X_train

# 3. Modeling 
## 3.1. Training Random Forest model and evaluation

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

## 3.2. Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

In [ ]:
y_pred_test = model.predict(X_test)
accuracy_score(y_test, y_pred_test)

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(data=confusion_matrix(y_true=y_test, y_pred=y_pred_test),
             columns=['predicted_negative', 'predicted_positive'],
             index=['real_negative', 'real_positive'])

## 3.3. Hyperparameter search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = RandomForestClassifier()

params = {
    'n_estimators': [100, 300],
    'max_depth': [30, 60, 120],
    'min_samples_leaf': [2, 4, 8, 16],
}

grid_search = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=4, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
model = grid_search.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(model.get_params())
print('ACC train set: {}'.format(accuracy_score(y_train, y_pred_train)))
print('ACC test set: {}'.format(accuracy_score(y_test, y_pred_test)))

## 3.4. Creating classifier Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
classifier_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(min_df=2, max_df=0.3)),
    ('classifier', RandomForestClassifier(**model.get_params()))
])

In [ ]:
classifier_pipeline.fit(cleaned_reviews,
                        dataset['sentiment'].values.ravel())

In [ ]:
raw_samples = ['This was the best movie i have ever seen. It should be in every cinema! I love it! :P ',
              'Terrible stuff, i cant say its good, what a waste of time... 0/10']
cleaned_samples = text_cleaner.transform(raw_samples)
classifier_pipeline.predict_proba(raw_samples)

In [ ]:
import joblib

joblib.dump(classifier_pipeline, 'classifier_pipeline.pkl')

In [ ]:
classifier_pipeline_restored = joblib.load('classifier_pipeline.pkl')

classifier_pipeline_restored.predict_proba(raw_samples)